In [1]:
import sys
import pickle
import numpy as np
import pandas as pd
#from nlp_utils import clean_text, pos_tag_text
sys.path.append("../")
from param_config import config
from sklearn.model_selection import KFold
from sklearn import metrics
import xgboost as xgb
import datetime

pd.set_option('display.float_format',lambda x: '%.5f'%x)


In [2]:
print("Load data...")

dfTrain = pd.read_csv(config.original_train_data_path)

dfPred= pd.read_csv(config.original_test_data_path)
dfPred2= pd.read_csv(config.original_test_data2_path)
# number of train/test samples
num_train, num_pred = dfTrain.shape[0], dfPred.shape[0]

print("Done.")

f = open('../../Data/col_name.txt','r')  
a = f.read()  
col_name = eval(a)  
f.close() 

f = open('../../Data/Procedure.txt','r')  
a = f.read()  
procedure = eval(a)  
f.close()

f = open('../../Cache/var_change.txt','r')
a = f.read()
var_change = eval(a)
f.close()

dfTrain = dfTrain.rename(columns=col_name)
raw_predictors = dfTrain.columns.tolist()[1:-1]
dfTrain = dfTrain.set_index(dfTrain['ID'])
dfPred = dfPred.rename(columns=col_name)
dfPred = dfPred.set_index(dfPred['ID'])
dfPred2 = dfPred2.rename(columns=col_name)

dfPredAll = pd.concat([dfPred,dfPred2],axis=0)
dfPredAll = dfPredAll.set_index(dfPredAll['ID'])

dfAll = pd.concat([dfTrain,dfPred,dfPred2],axis=0)

Load data...
Done.


In [3]:
#load features
Feat_file = ['Feat_cnt_col','Feat_cnt_tool','Feat_pcent_col','Feat_pcent_tool']
ex_feat = []
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv')
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    ex_feat += tmp_feat_list
    dfTrain = pd.merge(dfTrain,tmp,'left','ID')
    dfPred = pd.merge(dfPred,tmp,'left','ID')
   
        


In [ ]:
##define the variables
'''var_change = {'raw':raw}
for filename in Feat_file:
    tmp = pd.read_csv('../../Cache/'+filename+'.csv',nrows=10)
    tmp.set_index(tmp['ID'],inplace=True,drop=True)
    tmp_feat_list = tmp.columns.tolist()
    if 'ID' in tmp_feat_list:
        tmp_feat_list.remove('ID')
    var_change[filename] = {'constant':[],'category':[]}
    for var in tmp_feat_list:
        if dfTrain[var].nunique()==1 and dfTrain[var].isnull().sum()==0:
            var_change[filename]['constant'].append(var)
        elif dfTrain[var].nunique()<=5 or 'TOOL' in var:
            var_change[filename]['category'].append(var)
'''



In [4]:
#define the input variables, dummies
predictors = raw_predictors+ex_feat
for key,value in var_change.items():
    print(key)
    for var in var_change[key]['constant']:
        predictors.remove(var)
    for var in var_change[key]['category']:
        predictors.remove(var)
        tmpTrain = pd.get_dummies(dfTrain[var],prefix=var,dummy_na=True)
        tmpPred = pd.get_dummies(dfPred[var],prefix=var,dummy_na=True)
        predictors = predictors + tmpTrain.columns.tolist()
        dfTrain = pd.concat([dfTrain,tmpTrain],axis=1)
        dfPred = pd.concat([dfPred,tmpPred],axis=1)
for var in predictors:
    if var not in dfPred.columns:
        dfPred[var] = 0

raw
Feat_cnt_col
Feat_cnt_tool
Feat_pcent_col
Feat_pcent_tool


In [5]:
print(dfTrain[predictors].shape)
print(dfPred[predictors].shape)

(500, 58736)
(100, 58736)


In [12]:
def xgb_kfold(dfTrain,dfPred,predictors,n_splits=5):  
    kf = KFold(n_splits=n_splits,shuffle=True)
    dpred = xgb.DMatrix(dfPred[predictors].values,label=[0]*len(dfPred),missing=np.nan)
    imp = pd.DataFrame({'variable':predictors,'lk':['f'+str(i) for i in range(len(predictors))]})
    round=0
    for train_index, test_index in kf.split(dfTrain):
        round+=1
        train_X = dfTrain.loc[train_index,predictors]
        test_X = dfTrain.loc[test_index,predictors]
        train_Y = dfTrain.loc[train_index,'Y']
        test_Y = dfTrain.loc[test_index,'Y']

        dtrain = xgb.DMatrix(train_X.values, label=train_Y.values, missing = np.nan)
        dtest = xgb.DMatrix(test_X.values, label=test_Y.values, missing = np.nan)
        param = param = {'max_depth':4, 'eta':0.01, 'silent':0,'objective':'reg:linear','lambda':3,'subsample':0.8,
                         'colsample_bytree':0.8}
        evallist  = [(dtrain,'train'),(dtest,'eval')]  
        num_round = 5000
        evals_dict = {}
        model = xgb.train(param,dtrain,num_round, evallist,early_stopping_rounds=50,evals_result=evals_dict,verbose_eval =50)
        performance_df = pd.DataFrame(evals_dict['eval'])
        bst_tree = len(performance_df)-51
        pred_test = model.predict(dtest,ntree_limit =bst_tree)

        tmp_imp = pd.DataFrame(model.get_score(importance_type='gain'),index=['imp_fold%d'%round]).T
        tmp_imp['lk'] = tmp_imp.index
        imp = imp.merge(tmp_imp,'left','lk').fillna(0)


        pred_score = model.predict(dpred,ntree_limit =bst_tree)
        if round==1:
            test_result = pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})
            result = pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score})
        else:
            test_result = pd.concat([test_result,pd.DataFrame({'ID':dfTrain.loc[test_index,'ID'].values,'score':pred_test,'target':test_Y})],axis=0)
            result = result.merge(pd.DataFrame({'ID':dfPred['ID'],'Score_%d'%round:pred_score}),'inner','ID')
    print("Test MSE:",metrics.mean_squared_error(test_result['target'], test_result['score']))
    return test_result,result,imp

In [13]:
test_result,result,imp = xgb_kfold(dfTrain,dfPred,predictors)

[0]	train-rmse:2.33285	eval-rmse:2.3264
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't improved in 50 rounds.
[50]	train-rmse:1.42636	eval-rmse:1.42001
[100]	train-rmse:0.88203	eval-rmse:0.876213
[150]	train-rmse:0.556934	eval-rmse:0.558083
[200]	train-rmse:0.361284	eval-rmse:0.37381
[250]	train-rmse:0.243177	eval-rmse:0.273993
[300]	train-rmse:0.169976	eval-rmse:0.221457
[350]	train-rmse:0.123994	eval-rmse:0.195594
[400]	train-rmse:0.094886	eval-rmse:0.184763
[450]	train-rmse:0.075495	eval-rmse:0.179612
[500]	train-rmse:0.062208	eval-rmse:0.177411
[550]	train-rmse:0.052413	eval-rmse:0.176235
[600]	train-rmse:0.04513	eval-rmse:0.175533
[650]	train-rmse:0.039544	eval-rmse:0.175585
Stopping. Best iteration:
[635]	train-rmse:0.041172	eval-rmse:0.175475

[0]	train-rmse:2.32894	eval-rmse:2.34264
Multiple eval metrics have been passed: 'eval-rmse' will be used for early stopping.

Will train until eval-rmse hasn't impro

In [11]:
result['score']=result[['Score_1','Score_2','Score_3','Score_4','Score_5']].mean(axis=1)
submit = result[['ID','score']]
today = datetime.date.today().strftime('%Y-%m-%d')
submit.to_csv('../../Submission/submit_%s.csv'%today,header=False,index=False)

In [ ]:
0.0304606416852